In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from analysts_tools.redash_methods import *
    #Frubana libraries
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging

In [2]:
def update_segments(city,customers,customerSegmentId):
    url = f"https://{city}.frubana.com/api/v1/segments/massive/update?api-key=00_growth-team-zzz-qqq_001"

    payload = json.dumps({
    "customerSegmentId": customerSegmentId,
    "customersIds": customers
    })
    headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.text

In [3]:
users = get_fresh_query_result("https://internal-redash.federate.frubana.com/",213089,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
users[['customers']] = users[['customers']].astype(int)

200


In [4]:
users_eliminar_pro = get_fresh_query_result("https://internal-redash.federate.frubana.com/",206229,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
users_eliminar_pro[['user_id']] = users_eliminar_pro[['user_id']].astype(int)

200


In [5]:
for city in ['SPO','VCP','CWB','BHZ']:
    users_out = users_eliminar_pro.loc[users_eliminar_pro.region_code == city].user_id.unique()
    users_info = users.loc[users.city == city].reset_index(drop=True).copy()
    segmentos = users_info.loc[users_info.customers.isin(users_out)].segments.unique()
    
    if len(segmentos) > 0:
        for seg in segmentos:
            customers = users_info.loc[(users_info.segments == seg) & ~(users_info.customers.isin(users_out))].customers.unique().tolist()
            response = update_segments(city,customers,int(seg))
            
            print(f'IT RUNS OK FOR {city} FOR HOMES -> ' + response + ' ' + str(seg) + '|| TOTAL CUSTOMERS:' + str(len(customers)))

IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749631|| TOTAL CUSTOMERS:612
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749624|| TOTAL CUSTOMERS:477
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749630|| TOTAL CUSTOMERS:581
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749710|| TOTAL CUSTOMERS:927
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749713|| TOTAL CUSTOMERS:641
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749642|| TOTAL CUSTOMERS:386
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749625|| TOTAL CUSTOMERS:556
IT RUNS OK FOR SPO FOR HOMES -> The massive update process of Customer Segment was created 749715|| TOTAL CUSTOMERS:943
IT RUNS OK FOR SPO FOR HOMES -> The mass